#### Module Summary
> This module does coarse classing of the development data and reduces further features with < 3% Information Value. The cut-off value can be re-configured from the configuration file. \
> the coarse classing output of this module will need manual intervention to group 'similar' bins together for the next module - fine_classing
>
> **Input Files**
> 1. Developemnt and Validation data with reduced features as obtained form 01_feature_reduction module (pickle)
> 2. Feature Readuction Summary (xlsx)
>
> **Output Files**
> 1. Development & Validation data with reduced features (pickle)
> 2. Feature Reduction Summary (xlsx)
> 3. Coarse Classing Data (xlsx)

<h4> 2.1 Import Modules & Parameters

In [1]:
import logit_config as cfg
exec(open('module_imports.py').read())

<h4> 2.2 Import Data

In [2]:
with open(f"{outpath}/model_data.pickle", 'rb') as f:
    model_data_pckl_dict = pickle.load(f)
dev_data = copy.deepcopy(model_data_pckl_dict['dev_data_out1'])
val_data = copy.deepcopy(model_data_pckl_dict['val_data_out1'])
print(f"Development Data Shape: {dev_data.shape}")

Development Data Shape: (341, 20)


<h4> 2.3 Coarse Classing

In [3]:
# %%time

# Get Variable List for which coarse classing needs to be done
var_list = [x for x in dev_data.columns.tolist() if x not in non_pred_varlist]

# Coarse Classing
dev_data_woe = copy.deepcopy(dev_data)
coarse_classing_df = coarse_classing(dev_data, dev_data_woe, resp_var, var_list, special_val_list)

# Update the Character vars Grouping
char_varlist = [x for x in dev_data.select_dtypes(exclude=np.number).columns.tolist() if x not in non_pred_varlist]

if len(char_varlist) > 0:
    man_enc_map_df = create_cat_groups(dev_data, resp_var, char_varlist)
    coarse_classing_df = coarse_classing_df.merge(man_enc_map_df[['feature', 'class', 'grp']], left_on=['VAR_NAME', 'VAR_BINS'], right_on=['feature', 'class'], how='left')
    coarse_classing_df['FINE_BIN_NUM'] = np.where(coarse_classing_df['grp'].isnull(), coarse_classing_df['FINE_BIN_NUM'], coarse_classing_df['grp']).astype(int)
    coarse_classing_df.drop(['feature', 'class', 'grp'], axis=1, inplace=True)
coarse_classing_df.head()

100%|██████████| 18/18 [00:00<00:00, 19.86it/s]


,VAR_NAME,VAR_BINS,TOT_ACTS,ROWP_TOT,COUNT_RESP,PER_RESP,COUNT_NON_RESP,PER_NON_RESP,RAW_ODDS,LN_ODDS,INFO_VAL,CH_SQ,RESP_RATE,FINE_BIN_NUM
0,smoothness_mean,"(-inf, 0.07466]",18,0.052786,0,0.000,18,0.083333,0.000000,0.000000,0.000000,10.416667,0.000000,1
1,smoothness_mean,"(0.07466, 0.07948]",17,0.049853,2,0.016,15,0.069444,4.340278,1.467938,0.078453,4.536490,0.117647,2
2,smoothness_mean,"(0.07948, 0.08223]",17,0.049853,2,0.016,15,0.069444,4.340278,1.467938,0.078453,4.536490,0.117647,3
3,smoothness_mean,"(0.08223, 0.0842]",17,0.049853,4,0.032,13,0.060185,1.880787,0.631690,0.017804,1.261702,0.235294,4
4,smoothness_mean,"(0.0842, 0.08546]",17,0.049853,2,0.016,15,0.069444,4.340278,1.467938,0.078453,4.536490,0.117647,5


<h4> 2.4 Information Value Check

In [4]:
# Compute IV
iv_df = pd.DataFrame(coarse_classing_df.groupby('VAR_NAME')['INFO_VAL'].sum().rename('IV')).rename_axis('feature').reset_index().sort_values('IV', ascending=False).reset_index(drop=True)

# Drop Variables
# iv_drop_df = iv_df[iv_df['IV'] <= iv_cutoff]
iv_drop_df = iv_df[iv_df.index >= 100]
dev_data.drop(iv_drop_df['feature'].tolist(), axis=1, inplace=True)
print(f"Information Value cut-off being used: {iv_cutoff}\nVariables Dropped: {len(iv_drop_df['feature'].tolist())}. Updated Development Data Shape: {dev_data.shape}")

# Export Feature Dropout Reason
iv_drop_df['drop_reason'] = 'low_iv'
drop_reason_df = pd.read_excel(f'{outpath}/{feature_reduction_outfile}', sheet_name='Drop Reason Summary')
drop_reason_df = drop_reason_df.append(iv_drop_df[['feature', 'drop_reason']])
export_data_to_excel(iv_df, f'{outpath}/{feature_reduction_outfile}', sheetName='Information Value')
export_data_to_excel(drop_reason_df, f'{outpath}/{feature_reduction_outfile}', sheetName='Drop Reason Summary')

Information Value cut-off being used: 0.03
Variables Dropped: 0. Updated Development Data Shape: (341, 20)


<h4> 2.5 Export Coarse Classing result

In [8]:
# Save Updated Development & Validation Data
model_data_pckl_dict['dev_data_out2'] = dev_data
model_data_pckl_dict['val_data_out2'] = val_data[dev_data.columns.tolist()]
with open(f'{outpath}/model_data.pickle', 'wb') as f:
    pickle.dump(model_data_pckl_dict, f)
    
# Coarse Classing Output
coarse_classing_df = coarse_classing_df[coarse_classing_df['VAR_NAME'].isin(dev_data.columns.tolist())]
export_coarse_classing_data(coarse_classing_df, f"{outpath}/{dev_fname.split('.')[0]}_coarse_classing.xlsx", 'coarse_classing_result')